In [5]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import wandb
import numpy as np
import torch
import torch.nn as nn
import pickle
import xml.etree.ElementTree as ET
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    matthews_corrcoef
)
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
from accelerate import Accelerator
# Imports specific to the custom peft lora model
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType


ModuleNotFoundError: No module named 'numpy'

In [2]:

# Helper Functions and Data Preparation
def truncate_labels(labels, max_length):
    """Truncate labels to the specified max_length."""
    return [label[:max_length] for label in labels]

def compute_metrics(p):
    """Compute metrics for evaluation."""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    # Remove padding (-100 labels)
    predictions = predictions[labels != -100].flatten()
    labels = labels[labels != -100].flatten()
    
    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)
    
    # Compute precision, recall, F1 score, and AUC
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    auc = roc_auc_score(labels, predictions)
    
    # Compute MCC
    mcc = matthews_corrcoef(labels, predictions) 
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc, 'mcc': mcc} 

def compute_loss(model, inputs):
    """Custom compute_loss function."""
    logits = model(**inputs).logits
    labels = inputs["labels"]
    loss_fct = nn.CrossEntropyLoss(weight=class_weights)
    active_loss = inputs["attention_mask"].view(-1) == 1
    active_logits = logits.view(-1, model.config.num_labels)
    active_labels = torch.where(
        active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
    )
    loss = loss_fct(active_logits, active_labels)
    return loss


In [3]:

# Load the data from pickle files (replace with your local paths)
with open("../data/train_sequences_chunked_by_family.pkl", "rb") as f:
    train_sequences = pickle.load(f)

with open("../data/test_sequences_chunked_by_family.pkl", "rb") as f:
    test_sequences = pickle.load(f)

with open("../data/train_labels_chunked_by_family.pkl", "rb") as f:
    train_labels = pickle.load(f)

with open("../data/test_labels_chunked_by_family.pkl", "rb") as f:
    test_labels = pickle.load(f)

# make dataset smaller:
train_sequences = train_sequences[0:300]
test_sequences = test_sequences[0:100]
train_labels = train_labels[0:300]
test_labels = test_labels[0:100]

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")  # esm2_t12_35M_UR50D
max_sequence_length = 500 # 1000

train_tokenized = tokenizer(train_sequences, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)
test_tokenized = tokenizer(test_sequences, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)

# Directly truncate the entire list of labels
train_labels = truncate_labels(train_labels, max_sequence_length)
test_labels = truncate_labels(test_labels, max_sequence_length)

train_dataset = Dataset.from_dict({k: v for k, v in train_tokenized.items()}).add_column("labels", train_labels)
test_dataset = Dataset.from_dict({k: v for k, v in test_tokenized.items()}).add_column("labels", test_labels)



In [7]:
# Compute Class Weights
classes = np.array([0, 1])
flat_train_labels = [label for sublist in train_labels for label in sublist]
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=flat_train_labels)
accelerator = Accelerator()
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(accelerator.device)


In [8]:
# Define Custom Trainer Class
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = compute_loss(model, inputs)
        return (loss, outputs) if return_outputs else loss


In [12]:
# def train_function_no_sweeps(train_dataset, test_dataset):
    
#     # Set the LoRA config
#     config = {
#         "lora_alpha": 1, #try 0.5, 1, 2, ..., 16
#         "lora_dropout": 0.2,
#         "lr": 5.701568055793089e-04,
#         "lr_scheduler_type": "cosine",
#         "max_grad_norm": 0.5,
#         "num_train_epochs": 3,
#         "per_device_train_batch_size": 12,
#         "r": 2,
#         "weight_decay": 0.2,
#         # Add other hyperparameters as needed
#     }
#     # The base model you will train a LoRA on top of
#     model_checkpoint = "facebook/esm2_t6_8M_UR50D"  
    
#     # Define labels and model
#     id2label = {0: "No binding site", 1: "Binding site"}
#     label2id = {v: k for k, v in id2label.items()}
#     model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id)

#     # Convert the model into a PeftModel
#     peft_config = LoraConfig(
#         task_type=TaskType.TOKEN_CLS, 
#         inference_mode=False, 
#         r=config["r"], 
#         lora_alpha=config["lora_alpha"], 
#         target_modules=["query", "key", "value"], # also try "dense_h_to_4h" and "dense_4h_to_h"
#         lora_dropout=config["lora_dropout"], 
#         bias="none" # or "all" or "lora_only" 
#     )
#     model = get_peft_model(model, peft_config)

#     # Use the accelerator
#     model = accelerator.prepare(model)
#     train_dataset = accelerator.prepare(train_dataset)
#     test_dataset = accelerator.prepare(test_dataset)

#     timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

#     # Training setup
#     training_args = TrainingArguments(
#         output_dir=f"esm2_t12_35M-lora-binding-sites_{timestamp}",
#         learning_rate=config["lr"],
#         lr_scheduler_type=config["lr_scheduler_type"],
#         gradient_accumulation_steps=1,
#         max_grad_norm=config["max_grad_norm"],
#         per_device_train_batch_size=config["per_device_train_batch_size"],
#         per_device_eval_batch_size=config["per_device_train_batch_size"],
#         num_train_epochs=config["num_train_epochs"],
#         weight_decay=config["weight_decay"],
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="f1",
#         greater_is_better=True,
#         push_to_hub=False,
#         logging_dir=None,
#         logging_first_step=False,
#         logging_steps=200,
#         save_total_limit=7,
# #        no_cuda=False,
#         no_cuda=True,
#         seed=8893,
# #        fp16=True,
#         fp16=False,
#         report_to='wandb'
#     )

#     # Initialize Trainer
#     trainer = WeightedTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=test_dataset,
#         tokenizer=tokenizer,
#         data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
#         compute_metrics=compute_metrics
#     )

#     # Train and Save Model
#     trainer.train()
#     save_path = os.path.join("../results/lora_binding_sites", f"best_model_esm2_t12_35M_lora_{timestamp}")
#     trainer.save_model(save_path)
#     tokenizer.save_pretrained(save_path)


In [12]:
def train_function_no_sweeps(train_dataset, test_dataset):
    
    # Set the LoRA config
    config = {
        "lora_alpha": 1, #try 0.5, 1, 2, ..., 16
        "lora_dropout": 0.2,
        "lr": 5.701568055793089e-04,
        "lr_scheduler_type": "cosine",
        "max_grad_norm": 0.5,
        "num_train_epochs": 3,
        "per_device_train_batch_size": 12,
        "r": 2,
        "weight_decay": 0.2,
        # Add other hyperparameters as needed
    }
    # The base model you will train a LoRA on top of
    model_checkpoint = "facebook/esm2_t6_8M_UR50D"  # esm2_t12_35M_UR50D
    
    # Define labels and model
    id2label = {0: "No binding site", 1: "Binding site"}
    label2id = {v: k for k, v in id2label.items()}
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id)

    # Convert the model into a PeftModel
    peft_config = LoraConfig(
        task_type=TaskType.TOKEN_CLS, 
        inference_mode=False, 
        r=config["r"], 
        lora_alpha=config["lora_alpha"], 
        target_modules=["query", "key", "value"], # also try "dense_h_to_4h" and "dense_4h_to_h"
        lora_dropout=config["lora_dropout"], 
        bias="none" # or "all" or "lora_only" 
    )
    model = get_peft_model(model, peft_config)

    # Use the accelerator
    model = accelerator.prepare(model)
    train_dataset = accelerator.prepare(train_dataset)
    test_dataset = accelerator.prepare(test_dataset)

    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    # Training setup
    training_args = TrainingArguments(
        output_dir=f"esm2_t6_8M-lora_-binding-sites_{timestamp}",
        learning_rate=config["lr"],
        lr_scheduler_type=config["lr_scheduler_type"],
        gradient_accumulation_steps=1,
        max_grad_norm=config["max_grad_norm"],
        per_device_train_batch_size=config["per_device_train_batch_size"],
        per_device_eval_batch_size=config["per_device_train_batch_size"],
        num_train_epochs=config["num_train_epochs"],
        weight_decay=config["weight_decay"],
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        push_to_hub=False,
        logging_dir=None,
        logging_first_step=False,
        logging_steps=200,
        save_total_limit=7,
        no_cuda=False,
#        no_cuda=True,
        seed=8893,
        fp16=True,
#        fp16=False,
#        report_to='wandb'
    )

    # Initialize Trainer
    trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
        compute_metrics=compute_metrics
    )

    # Train and Save Model
    trainer.train()
    save_path = os.path.join("../results/lora_binding_sites", f"best_model_esm2_t6_8M_lora_{timestamp}")
    trainer.save_model(save_path)
    tokenizer.save_pretrained(save_path)


In [ ]:
train_function_no_sweeps(train_dataset, test_dataset)

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/elisa.gomezdelope/Library/CloudStorage/OneDrive-UniversityofLuxembourg/laptop_bck/Miscellaneous projects/GSoC24/ESMbind/code/.venv/esmbind/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/elisa.gomezdelope/Library/CloudStorage/OneDrive-UniversityofLuxembourg/laptop_bck/Miscellaneous projects/GSoC24/ESMbind/code/.venv/esmbind/lib/python3.12/site-packages/transformers/training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warni

In [ ]:
from sklearn.metrics import(
    matthews_corrcoef, 
    accuracy_score, 
    precision_recall_fscore_support, 
    roc_auc_score
)
from peft import PeftModel
from transformers import DataCollatorForTokenClassification


In [ ]:
# Define paths to the LoRA and base models
base_model_path = "facebook/esm2_t6_8M_UR50D"  # esm2_t12_35M_UR50D"
lora_model_path = "../results/lora_binding_sites/best_model_esm2_t6_8M_lora_2024-05-25_19-32-55" # "path/to/your/lora/model" Replace with the correct path to your LoRA model

# Load the base model
base_model = AutoModelForTokenClassification.from_pretrained(base_model_path)

# Load the LoRA model
model = PeftModel.from_pretrained(base_model, lora_model_path)
model = accelerator.prepare(model)  # Prepare the model using the accelerator


In [ ]:

# Define label mappings
id2label = {0: "No binding site", 1: "Binding site"}
label2id = {v: k for k, v in id2label.items()}

# Create a data collator
data_collator = DataCollatorForTokenClassification(tokenizer)


In [ ]:
# Define a function to compute the metrics
def compute_metrics(dataset):
    # Get the predictions using the trained model
    trainer = Trainer(model=model, data_collator=data_collator)
    predictions, labels, _ = trainer.predict(test_dataset=dataset)
    
    # Remove padding and special tokens
    mask = labels != -100
    true_labels = labels[mask].flatten()
    flat_predictions = np.argmax(predictions, axis=2)[mask].flatten().tolist()

    # Compute the metrics
    accuracy = accuracy_score(true_labels, flat_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, flat_predictions, average='binary')
    auc = roc_auc_score(true_labels, flat_predictions)
    mcc = matthews_corrcoef(true_labels, flat_predictions)  # Compute the MCC
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "auc": auc, "mcc": mcc}  # Include the MCC in the returned dictionary


In [ ]:
# Get the metrics for the training and test datasets
train_metrics = compute_metrics(train_dataset)
test_metrics = compute_metrics(test_dataset)

train_metrics, test_metrics

# Inference

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from peft import PeftModel
import torch

# Path to the saved LoRA model and ESM2 base model
base_model_path = "facebook/esm2_t6_8M_UR50D"  # esm2_t12_35M_UR50D"
model_path = "../results/lora_binding_sites/best_model_esm2_t6_8M_lora_2024-05-25_19-32-55" # "path/to/your/lora/model" Replace with the correct path to your LoRA model

# Load the model
base_model = AutoModelForTokenClassification.from_pretrained(base_model_path)
loaded_model = PeftModel.from_pretrained(base_model, model_path)

# Ensure the model is in evaluation mode
loaded_model.eval()

# Load the tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained(base_model_path)


In [ ]:

# Protein sequence for inference
protein_sequence = "MAVPETRPNHTIYINNLNEKIKKDELKKSLHAIFSRFGQILDILVSRSLKMRGQAFVIFKEVSSATNALRSMQGFPFYDKPMRIQYAKTDSDIIAKMKGT"  # Replace with your actual sequence

# Tokenize the sequence
inputs = loaded_tokenizer(protein_sequence, return_tensors="pt", truncation=True, max_length=1024, padding='max_length')

# Run the model
with torch.no_grad():
    logits = loaded_model(**inputs).logits

# Get predictions
tokens = loaded_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])  # Convert input ids back to tokens
predictions = torch.argmax(logits, dim=2)

# Define labels
id2label = {
    0: "No binding site",
    1: "Binding site"
}

# Print the predicted labels for each token
for token, prediction in zip(tokens, predictions[0].numpy()):
    if token not in ['<pad>', '<cls>', '<eos>']:
        print((token, id2label[prediction]))
